In [1]:
!pip install polars

In [41]:
import polars as pl

In [42]:
euka = pl.read_parquet('/home/onyxia/work/KEGG_Pipeline/eukaryotes/hsa_genome.parquet')

In [43]:
euka


shape: (2_469, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ gene_id                ┆ K0                     ┆ Dna_SEQ                ┆ Prot                  │
│ ---                    ┆ ---                    ┆ ---                    ┆ ---                   │
│ list[str]              ┆ list[str]              ┆ list[str]              ┆ list[bool]            │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ ["hsa:102466751",      ┆ ["", "", … "K04257"]   ┆ ["gagaggaacatgggctcagg ┆ [false, false, …      │
│ "hsa:100302278…        ┆                        ┆ aca", "ttg…            ┆ true]                 │
│ ["hsa:148398",         ┆ ["", "K14833", … ""]   ┆ ["atgtccaaggggatcctgca ┆ [true, true, … true]  │
│ "hsa:26155", … "h…     ┆                        ┆ ggtgcatcct…            ┆                       │
│ ["hsa:406984",         ┆ ["K17016", "K17015", … ┆ ["catcttactgggcagcattg ┆ [false, false, …      │
│ "hsa:406983", … "…     ┆ "K04554"]              ┆ ga", "catc…            ┆ true]                 │
│ ["hsa:6339",           ┆ ["K04826", "K12489", … ┆ ["atgagggcagtgctgtcaca ┆ [true, true, … true]  │
│ "hsa:116983", … "hs…   ┆ "K02353"]              ┆ gaagacaaca…            ┆                       │
│ …                      ┆ …                      ┆ …                      ┆ …                     │
│ ["hsa:124908536",      ┆ ["", "", … ""]         ┆ ["atggctccgtctgctgctgt ┆ [true, true, … true]  │
│ "hsa:124908905…        ┆                        ┆ accagaccgg…            ┆                       │
│ ["hsa:124909094",      ┆ ["", "", … ""]         ┆ ["atggcgccatctttgctcac ┆ [true, true, … true]  │
│ "hsa:128966656…        ┆                        ┆ tgcaacatcg…            ┆                       │
│ ["hsa:124909329",      ┆ ["", "", … ""]         ┆ ["atgcgccctgagggctcgct ┆ [true, true, … true]  │
│ "hsa:124909330…        ┆                        ┆ gacctaccgg…            ┆                       │
│ ["hsa:130160549",      ┆ ["", "", … ""]         ┆ ["atggagaccggcaggcaaac ┆ [true, true, … true]  │
│ "hsa:130160544…        ┆                        ┆ aggtgtgtct…            ┆                       │
└────────────────────────┴────────────────────────┴────────────────────────┴───────────────────────┘

In [45]:
import os 
print(len(os.listdir("/home/onyxia/work/KEGG_Pipeline/eukaryotes/")))


614


In [12]:
import requests
def get_eukaryotic_organisms():
    # Define the KEGG API URL for all organisms
    kegg_api_url = "http://rest.kegg.jp/list/organism"
    
    try:
        # Send a GET request to the KEGG API
        response = requests.get(kegg_api_url)
        
        # Check if the request was successful (HTTP status code 200)
        if response.status_code == 200:
            # Split the response text into lines
            lines = response.text.strip().split('\n')
            # Extract the organism codes and names from the lines
            organisms = [line.split('\t') for line in lines]
            # Filter for prokaryotic organisms (Bacteria and Archaea)
            euka_organisms = [org for org in organisms if "Eukaryotes" in org[3]]

            return euka_organisms
        else:
            from bs4 import BeautifulSoup
            print(f"Error: Failed to retrieve data from KEGG API. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error: {e}")


In [13]:
euka_genomes = get_eukaryotic_organisms()

In [14]:
len(euka_genomes)

999

In [46]:
import s3fs
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},key ='YZ95UR2UJY1LVYJLCO5C', secret = '00trAv4o6IwI9rhzBPOt9zNoBGTCVBq5WvK3hA35', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJZWjk1VVIyVUpZMUxWWUpMQ081QyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzA3MjI1MTcwLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImZsb3JpYW4uYmFzc29AZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzA3MzM2NjEwLCJmYW1pbHlfbmFtZSI6IkJhc3NvIiwiZ2l2ZW5fbmFtZSI6IkZsb3JpYW4iLCJncm91cHMiOlsiVVNFUl9PTllYSUEiXSwiaWF0IjoxNzA3MjUwMjEwLCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiYWFiMGViYjktMmNlOS00MTBiLWJlZjgtZTdhNmI0ODZmNTg5IiwibmFtZSI6IkZsb3JpYW4gQmFzc28iLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2FtZXIzNSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvbl9zdGF0ZSI6Ijc3ZmZlNDYyLWI1MTgtNDQyNS05MTUzLTc3NTM4ZDlmOWI1NyIsInNpZCI6Ijc3ZmZlNDYyLWI1MTgtNDQyNS05MTUzLTc3NTM4ZDlmOWI1NyIsInN1YiI6IjVmY2JjNGE5LThmNzItNDkzMy05MDJmLTNiYWNlMjMyNjk2MSIsInR5cCI6IkJlYXJlciJ9.pYNUuSPMwrMIQY5x_095HWKlFzsDIhebyE16mw9O0CUzZ-puvKlh_vhPQR-OJ_p7HIKkeO5XfSxY_rF49eqMQQ')

In [47]:
import os
euka_path='/home/onyxia/work/KEGG_Pipeline/eukaryotes/'
s3_folder='gamer35/KEGG_db/genomes_eukaryote/'


In [48]:
for subdir, dirs, files in os.walk(euka_path):
    for file in files:
        genome_path = os.path.join(subdir, file)
        fs.put(genome_path, s3_folder)
        print(f'{file} uploaded')


dnm_genome.parquet uploaded
bcoo_genome.parquet uploaded
pruf_genome.parquet uploaded
phu_genome.parquet uploaded
ttt_genome.parquet uploaded
cbrc_genome.parquet uploaded
tfn_genome.parquet uploaded
daz_genome.parquet uploaded
pbi_genome.parquet uploaded
ccae_genome.parquet uploaded
kaf_genome.parquet uploaded
oga_genome.parquet uploaded
otc_genome.parquet uploaded
mgen_genome.parquet uploaded
tdl_genome.parquet uploaded
cmac_genome.parquet uploaded
ctig_genome.parquet uploaded
fas_genome.parquet uploaded
pavi_genome.parquet uploaded
svg_genome.parquet uploaded
gas_genome.parquet uploaded
manu_genome.parquet uploaded
pps_genome.parquet uploaded
tgb_genome.parquet uploaded
spaa_genome.parquet uploaded
cpea_genome.parquet uploaded
cten_genome.parquet uploaded
tmn_genome.parquet uploaded
dpl_genome.parquet uploaded
nlu_genome.parquet uploaded
pee_genome.parquet uploaded
slal_genome.parquet uploaded
tala_genome.parquet uploaded
cqd_genome.parquet uploaded
ppot_genome.parquet uploaded
zvi_g